###Function -1

In [ ]:
#Importing libraries and saved model ,files
import pandas as pd
import pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score,confusion_matrix,accuracy_score

Pkl_Filename = "/content/drive/My Drive/Self Case study -1/median_values.pkl" 
with open(Pkl_Filename, 'rb') as file:  
    med_values= pickle.load(file)

Pkl_Filename = "/content/drive/My Drive/Self Case study -1/norm_min.pkl" 
with open(Pkl_Filename, 'rb') as file:  
    norm_min= pickle.load(file)

Pkl_Filename = "/content/drive/My Drive/Self Case study -1/norm_max.pkl" 
with open(Pkl_Filename, 'rb') as file:  
    norm_max= pickle.load(file)

Pkl_Filename = "/content/drive/My Drive/Self Case study -1/XGBoost_std.pkl"  
# Load the Model back from file
with open(Pkl_Filename, 'rb') as file:  
    XGBoost_imp= pickle.load(file)

In [ ]:
#reading raw data 
raw_data=pd.read_csv('/content/drive/My Drive/Self Case study -1/equip_failures_training_set.csv')
raw_data.head(4)

,id,target,sensor1_measure,sensor2_measure,sensor3_measure,sensor4_measure,sensor5_measure,sensor6_measure,sensor7_histogram_bin0,sensor7_histogram_bin1,sensor7_histogram_bin2,sensor7_histogram_bin3,sensor7_histogram_bin4,sensor7_histogram_bin5,sensor7_histogram_bin6,sensor7_histogram_bin7,sensor7_histogram_bin8,sensor7_histogram_bin9,sensor8_measure,sensor9_measure,sensor10_measure,sensor11_measure,sensor12_measure,sensor13_measure,sensor14_measure,sensor15_measure,sensor16_measure,sensor17_measure,sensor18_measure,sensor19_measure,sensor20_measure,sensor21_measure,sensor22_measure,sensor23_measure,sensor24_histogram_bin0,sensor24_histogram_bin1,sensor24_histogram_bin2,sensor24_histogram_bin3,sensor24_histogram_bin4,sensor24_histogram_bin5,...,sensor77_measure,sensor78_measure,sensor79_measure,sensor80_measure,sensor81_measure,sensor82_measure,sensor83_measure,sensor84_measure,sensor85_measure,sensor86_measure,sensor87_measure,sensor88_measure,sensor89_measure,sensor90_measure,sensor91_measure,sensor92_measure,sensor93_measure,sensor94_measure,sensor95_measure,sensor96_measure,sensor97_measure,sensor98_measure,sensor99_measure,sensor100_measure,sensor101_measure,sensor102_measure,sensor103_measure,sensor104_measure,sensor105_histogram_bin0,sensor105_histogram_bin1,sensor105_histogram_bin2,sensor105_histogram_bin3,sensor105_histogram_bin4,sensor105_histogram_bin5,sensor105_histogram_bin6,sensor105_histogram_bin7,sensor105_histogram_bin8,sensor105_histogram_bin9,sensor106_measure,sensor107_measure
0,1,0,76698,na,2130706438,280,0,0,0,0,0,0,37250,1432864,3664156,1007684,25896,0,2551696,0,0,0,0,0,4933296,3655166,1766008,1132040,0,0,0,0,1012,268,0,0,0,0,0,469014,...,18,5330690,4732,1126,0,0,0,0,0,0,0,0,62282,85908,32790,0,0,202710,37928,14745580,1876644,0,0,0,0,2801180,2445.8,2712,965866,1706908,1240520,493384,721044,469792,339156,157956,73224,0,0,0
1,2,0,33058,na,0,na,0,0,0,0,0,0,18254,653294,1720800,516724,31642,0,1393352,0,68,0,0,0,2560898,2127150,1084598,338544,0,0,0,0,0,0,0,0,0,0,0,71510,...,na,na,3312,522,0,0,0,0,0,0,0,0,33736,36946,5936,0,0,103330,16254,4510080,868538,0,0,0,0,3477820,2211.76,2334,664504,824154,421400,178064,293306,245416,133654,81140,97576,1500,0,0
2,3,0,41040,na,228,100,0,0,0,0,0,0,1648,370592,1883374,292936,12016,0,1234132,0,0,0,0,0,2371990,2173634,300796,153698,0,0,0,0,358,110,0,0,0,0,0,0,...,0,2341048,1494,152,0,0,0,0,0,0,0,0,13876,38182,8138,0,0,65772,10534,300240,48028,0,0,0,0,1040120,1018.64,1020,262032,453378,277378,159812,423992,409564,320746,158022,95128,514,0,0
3,4,0,12,0,70,66,0,10,0,0,0,318,2212,3232,1872,0,0,0,2668,0,0,0,642,3894,10184,7554,10764,1014,0,0,0,0,60,6,0,0,0,0,0,0,...,0,2578,76,62,0,0,0,0,0,0,0,0,232,0,0,2014,370,48,18,15740,1822,20174,44,0,0,0,1.08,54,5670,1566,240,46,58,44,10,0,0,0,4,32


In [ ]:
#splitting into test and train for future checking
y=raw_data[['target']]
X=raw_data.drop(['target'],axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

In [ ]:
#random single data point
X=raw_data.iloc[7869,:]

#its target to check
y=X['target']
print('target : ',y)

target :  0


## Function 1:

In [ ]:
def  final_fun_1(X):
        
    X=X.replace('na',np.NaN)
    X=X.astype(float)

    red_feat=['sensor2_measure','sensor40_measure','sensor41_measure','sensor42_measure','sensor43_measure','sensor68_measure',
                'sensor7_histogram_bin0','sensor7_histogram_bin1','sensor19_measure','sensor21_measure','sensor24_histogram_bin0',
                'sensor24_histogram_bin1','sensor24_histogram_bin2','sensor24_histogram_bin3','sensor24_histogram_bin4',
                'sensor24_histogram_bin9','sensor25_histogram_bin9','sensor54_measure','sensor64_histogram_bin0','sensor106_measure']

    to_drop_col=['sensor6_measure', 'sensor7_histogram_bin4', 'sensor7_histogram_bin7', 'sensor8_measure', 'sensor11_measure',
                    'sensor13_measure', 'sensor14_measure', 'sensor15_measure', 'sensor16_measure', 'sensor17_measure', 'sensor24_histogram_bin3', 
                    'sensor25_histogram_bin5', 'sensor26_histogram_bin0', 'sensor26_histogram_bin1', 'sensor26_histogram_bin2', 'sensor26_histogram_bin3',
                    'sensor26_histogram_bin4', 'sensor26_histogram_bin5', 'sensor26_histogram_bin6', 'sensor27_measure', 'sensor32_measure', 
                    'sensor33_measure', 'sensor34_measure', 'sensor35_measure', 'sensor37_measure', 'sensor38_measure', 'sensor39_measure', 'sensor40_measure',
                    'sensor41_measure', 'sensor42_measure', 'sensor43_measure', 'sensor45_measure', 'sensor46_measure', 'sensor47_measure', 'sensor48_measure',
                    'sensor49_measure', 'sensor52_measure', 'sensor53_measure', 'sensor56_measure', 'sensor59_measure', 'sensor61_measure', 
                    'sensor64_histogram_bin1', 'sensor64_histogram_bin2', 'sensor64_histogram_bin3', 'sensor64_histogram_bin4', 'sensor64_histogram_bin5', 
                    'sensor64_histogram_bin6', 'sensor65_measure', 'sensor67_measure', 'sensor69_histogram_bin0', 'sensor69_histogram_bin2', 
                    'sensor69_histogram_bin3', 'sensor69_histogram_bin5', 'sensor73_measure', 'sensor75_measure', 'sensor78_measure', 'sensor88_measure',
                    'sensor89_measure', 'sensor91_measure', 'sensor94_measure', 'sensor95_measure', 'sensor97_measure', 'sensor103_measure', 'sensor104_measure',
                    'sensor105_histogram_bin0', 'sensor105_histogram_bin1', 'sensor105_histogram_bin2', 'sensor105_histogram_bin3', 'sensor105_histogram_bin4']


    final_feat=['sensor1_measure', 'sensor3_measure', 'sensor4_measure','sensor7_histogram_bin2', 'sensor7_histogram_bin3',
        'sensor7_histogram_bin8', 'sensor9_measure', 'sensor10_measure','sensor12_measure', 'sensor18_measure', 'sensor20_measure',
        'sensor22_measure', 'sensor23_measure', 'sensor24_histogram_bin5','sensor24_histogram_bin6', 'sensor24_histogram_bin7',
        'sensor24_histogram_bin8', 'sensor25_histogram_bin0','sensor25_histogram_bin1', 'sensor25_histogram_bin2','sensor25_histogram_bin3',
        'sensor25_histogram_bin6','sensor25_histogram_bin8', 'sensor26_histogram_bin8','sensor26_histogram_bin9', 'sensor28_measure', 'sensor29_measure',
        'sensor31_measure', 'sensor36_measure', 'sensor44_measure','sensor50_measure', 'sensor51_measure', 'sensor55_measure',
        'sensor57_measure', 'sensor60_measure', 'sensor62_measure','sensor63_measure', 'sensor66_measure', 'sensor69_histogram_bin4',
        'sensor69_histogram_bin8', 'sensor69_histogram_bin9','sensor77_measure', 'sensor81_measure', 'sensor82_measure',
        'sensor92_measure', 'sensor96_measure', 'sensor98_measure','sensor99_measure', 'sensor102_measure', 'sensor105_histogram_bin7',
        'sensor105_histogram_bin9', 's_hist_7', 's_hist_24', 's_hist_64','mean', 'std']

    to_drop_col=list(set(to_drop_col) - set(red_feat))
    X=pd.DataFrame(X).transpose()
    X=X.drop(['target'],axis=1)
    #dropping 'id' column from the dataset
    X=X.drop(['id'],axis=1)
    X= X.drop(red_feat,axis=1)
    X=X.drop(to_drop_col, axis=1)

    for col in X.columns:
        if col not in ['id','target']:
            X[col] = X[col].fillna(med_values[col])

    l=len(X.columns)

    new_hist=[]
    for i in X.columns:
        if 'histogram_bin' in i:
            new_hist.append(i)

    #seggregating th features
    s_hist_7=new_hist[0:6]
    s_hist_24=new_hist[6:10]
    s_hist_25=new_hist[10:18]
    s_hist_26=new_hist[18:21]
    s_hist_64=new_hist[21:24]
    s_hist_69=new_hist[24:30]
    s_hist_105=new_hist[30:]

    all_hist=[s_hist_7,s_hist_24,s_hist_25,s_hist_26,s_hist_64,s_hist_69,s_hist_105]
    all_hist_names=['s_hist_7','s_hist_24','s_hist_25','s_hist_26','s_hist_64','s_hist_69','s_hist_105']

    for i in range(len(all_hist)):
        X[all_hist_names[i]]=X[all_hist[i]].mean(axis=1)

    X['mean']=X.iloc[:,1:l].mean(axis=1)
    X['std']=X.iloc[:,1:l].std(axis=1)
    X['sum']=X.iloc[:,1:l].sum(axis=1)

    for col in X.columns:
        n=X[col]-norm_min[col]
        d=norm_max[col]-norm_min[col]
        X[col]=n/d

    X=X[final_feat]

    return ('Target : ',XGBoost_imp.predict(X)[0])

#### Function 2:

In [ ]:
def  final_fun_2(X,y_test):
        
    X=X.replace('na',np.NaN)
    X=X.astype(float)

    red_feat=['sensor2_measure','sensor40_measure','sensor41_measure','sensor42_measure','sensor43_measure','sensor68_measure',
                'sensor7_histogram_bin0','sensor7_histogram_bin1','sensor19_measure','sensor21_measure','sensor24_histogram_bin0',
                'sensor24_histogram_bin1','sensor24_histogram_bin2','sensor24_histogram_bin3','sensor24_histogram_bin4',
                'sensor24_histogram_bin9','sensor25_histogram_bin9','sensor54_measure','sensor64_histogram_bin0','sensor106_measure']

    to_drop_col=['sensor6_measure', 'sensor7_histogram_bin4', 'sensor7_histogram_bin7', 'sensor8_measure', 'sensor11_measure',
                    'sensor13_measure', 'sensor14_measure', 'sensor15_measure', 'sensor16_measure', 'sensor17_measure', 'sensor24_histogram_bin3', 
                    'sensor25_histogram_bin5', 'sensor26_histogram_bin0', 'sensor26_histogram_bin1', 'sensor26_histogram_bin2', 'sensor26_histogram_bin3',
                    'sensor26_histogram_bin4', 'sensor26_histogram_bin5', 'sensor26_histogram_bin6', 'sensor27_measure', 'sensor32_measure', 
                    'sensor33_measure', 'sensor34_measure', 'sensor35_measure', 'sensor37_measure', 'sensor38_measure', 'sensor39_measure', 'sensor40_measure',
                    'sensor41_measure', 'sensor42_measure', 'sensor43_measure', 'sensor45_measure', 'sensor46_measure', 'sensor47_measure', 'sensor48_measure',
                    'sensor49_measure', 'sensor52_measure', 'sensor53_measure', 'sensor56_measure', 'sensor59_measure', 'sensor61_measure', 
                    'sensor64_histogram_bin1', 'sensor64_histogram_bin2', 'sensor64_histogram_bin3', 'sensor64_histogram_bin4', 'sensor64_histogram_bin5', 
                    'sensor64_histogram_bin6', 'sensor65_measure', 'sensor67_measure', 'sensor69_histogram_bin0', 'sensor69_histogram_bin2', 
                    'sensor69_histogram_bin3', 'sensor69_histogram_bin5', 'sensor73_measure', 'sensor75_measure', 'sensor78_measure', 'sensor88_measure',
                    'sensor89_measure', 'sensor91_measure', 'sensor94_measure', 'sensor95_measure', 'sensor97_measure', 'sensor103_measure', 'sensor104_measure',
                    'sensor105_histogram_bin0', 'sensor105_histogram_bin1', 'sensor105_histogram_bin2', 'sensor105_histogram_bin3', 'sensor105_histogram_bin4']


    final_feat=['sensor1_measure', 'sensor3_measure', 'sensor4_measure','sensor7_histogram_bin2', 'sensor7_histogram_bin3',
        'sensor7_histogram_bin8', 'sensor9_measure', 'sensor10_measure','sensor12_measure', 'sensor18_measure', 'sensor20_measure',
        'sensor22_measure', 'sensor23_measure', 'sensor24_histogram_bin5','sensor24_histogram_bin6', 'sensor24_histogram_bin7',
        'sensor24_histogram_bin8', 'sensor25_histogram_bin0','sensor25_histogram_bin1', 'sensor25_histogram_bin2','sensor25_histogram_bin3',
        'sensor25_histogram_bin6','sensor25_histogram_bin8', 'sensor26_histogram_bin8','sensor26_histogram_bin9', 'sensor28_measure', 'sensor29_measure',
        'sensor31_measure', 'sensor36_measure', 'sensor44_measure','sensor50_measure', 'sensor51_measure', 'sensor55_measure',
        'sensor57_measure', 'sensor60_measure', 'sensor62_measure','sensor63_measure', 'sensor66_measure', 'sensor69_histogram_bin4',
        'sensor69_histogram_bin8', 'sensor69_histogram_bin9','sensor77_measure', 'sensor81_measure', 'sensor82_measure',
        'sensor92_measure', 'sensor96_measure', 'sensor98_measure','sensor99_measure', 'sensor102_measure', 'sensor105_histogram_bin7',
        'sensor105_histogram_bin9', 's_hist_7', 's_hist_24', 's_hist_64','mean', 'std']

    to_drop_col=list(set(to_drop_col) - set(red_feat))
    #dropping 'id' column from the dataset
    X=X.drop(['id'],axis=1)
    X= X.drop(red_feat,axis=1)
    X=X.drop(to_drop_col, axis=1)

    for col in X.columns:
        X[col] = X[col].fillna(med_values[col])

    l=len(X.columns)

    new_hist=[]
    for i in X.columns:
        if 'histogram_bin' in i:
            new_hist.append(i)

    #seggregating th features
    s_hist_7=new_hist[0:6]
    s_hist_24=new_hist[6:10]
    s_hist_25=new_hist[10:18]
    s_hist_26=new_hist[18:21]
    s_hist_64=new_hist[21:24]
    s_hist_69=new_hist[24:30]
    s_hist_105=new_hist[30:]

    all_hist=[s_hist_7,s_hist_24,s_hist_25,s_hist_26,s_hist_64,s_hist_69,s_hist_105]
    all_hist_names=['s_hist_7','s_hist_24','s_hist_25','s_hist_26','s_hist_64','s_hist_69','s_hist_105']

    for i in range(len(all_hist)):
        X[all_hist_names[i]]=X[all_hist[i]].mean(axis=1)

    X['mean']=X.iloc[:,1:l].mean(axis=1)
    X['std']=X.iloc[:,1:l].std(axis=1)
    X['sum']=X.iloc[:,1:l].sum(axis=1)

    for col in X.columns:
        n=X[col]-norm_min[col]
        d=norm_max[col]-norm_min[col]
        X[col]=n/d

    X=X[final_feat]
    pred_y_test=XGBoost_imp.predict(X)
    
    return ('F1 Score : ', f1_score(y_test, pred_y_test, average='binary'))

### Prediction using function 1 and function 2

In [ ]:
final_fun_1(X)

('Target : ', 0.0)

In [ ]:
final_fun_2(X_test,y_test)

('F1 Score : ', 0.9611650485436893)